This implementation performs the following changes:

1. Converts 'datetime' timestamp datatype to string datatype 'datetime_str'
> a. Currently not included in VectorAssembler due to overfitting

2. Original feature columns implemented
> a.  'local_resp', 'resp_ip_bytes', 'ts', 'datetime_str' currently not included in VectorAssembler due to overfitting



In [ ]:
!pip install numpy
!pip install pyspark
!pip install pandas

In [16]:
#Basic Imports
import pyspark
from pyspark.sql import SparkSession

#ML Classifier Imports
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import OneVsRest
from pyspark.ml import Pipeline
from pyspark.sql.functions import when
import numpy as np
import pandas as pd
from pyspark.sql.functions import col

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("rka7") \
    .master("local") \
    .config("spark.driver.cores", "5") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "5g") \
    .config("spark.executor.cores", "4") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "2") \
    .config("spark.dynamicAllocation.maxExecutors", "4") \
    .config("spark.executor.instances", "2") \
.getOrCreate()

In [18]:
# Get the parquet files (current example is 2 from the website)
parquet_files = ["/content/part-00000-7c2e9adb-5430-4792-a42b-10ff5bbd46e8-c000.snappy.parquet", \
                 "/content/part-00000-df678a79-4a73-452b-8e72-d624b2732f17-c000.snappy.parquet"]
# Read the parquet files into a dataframe
df = spark.read.parquet(*parquet_files, inferSchema=True)

In [ ]:
# Remove instances with null values
df = df.dropna()
#df.printSchema()

In [20]:
# Convert timestamp column to string
#df = df.withColumn("datetime_str", col("datetime").cast("string"))

#Drop the datetime column
df = df.drop("datetime")

# Define columns to index
columns_to_index = ['service', 'conn_state', 'history', 'proto', 'dest_ip_zeek', 'community_id', 'uid', 'src_ip_zeek', 'label_tactic']

# Apply StringIndexer to each column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed").fit(df) for column in columns_to_index]

# Chain indexers together
pipeline = Pipeline(stages=indexers)

# Fit and transform the data
df_indexed = pipeline.fit(df).transform(df)

# Drop original columns
df_indexed = df_indexed.drop(*columns_to_index)

# Drop rows with any null values
df_indexed = df_indexed.dropna()

# Show the schema of the DataFrame
#df_indexed.printSchema()
#df.select("label_tactic").distinct().collect()

In [21]:
#df_assembled = df_assembled.drop("datetime_str")

In [ ]:
# List of columns to assemble
columns_to_assemble = df_indexed.columns

# Include Numerical features to be assembled (Problems: 'local_resp', 'resp_ip_bytes', 'ts' is causing evaluations to overfit?)
columns_to_assemble.extend(['resp_pkts', 'orig_ip_bytes', 'missed_bytes', 'duration', 'orig_pkts', \
               'dest_port_zeek', 'orig_bytes', 'local_orig', 'resp_bytes', 'src_port_zeek'])

# Remove the target column (label) if it's in the list
columns_to_assemble.remove('label_tactic_indexed')

# Create the VectorAssembler
assembler = VectorAssembler(inputCols=columns_to_assemble, outputCol="features")

# Transform the DataFrame
df_assembled = assembler.transform(df_indexed)

# Select only the features and label columns
df_assembled = df_assembled.select("features", "label_tactic_indexed")

# Show the schema of the DataFrame
#df_assembled.printSchema()

In [ ]:
# Split the data into training and test sets
train_data, test_data = df_assembled.randomSplit([0.8, 0.2], seed=1)

# Create the SVM model
svm = LinearSVC(labelCol="label_tactic_indexed", featuresCol="features", maxIter=10)

# One Vs. Rest
ovr = OneVsRest(classifier=svm, labelCol='label_tactic_indexed')

# Fit the model
svm_model = ovr.fit(train_data)

# Make predictions
predictions = svm_model.transform(test_data)

In [ ]:
#Creating Mulitple Evaluators for weighted precision, weighted recall, accuracy, weighted FPR
eval_precision = MulticlassClassificationEvaluator(labelCol="label_tactic_indexed", metricName="weightedPrecision")
eval_recall = MulticlassClassificationEvaluator(labelCol="label_tactic_indexed", metricName="weightedRecall")
#eval_accuracy = MulticlassClassificationEvaluator(labelCol="label_tactic_indexed", metricName="accuracy")
#eval_fpr = MulticlassClassificationEvaluator(labelCol="label_tactic_indexed", metricName="weightedFalsePositiveRate")

# Evaluate the model
evaluator = eval_accuracy.evaluate(predictions)
print("Accuracy:", evaluator)

Changes made:

Numerical columns included in VectorAssembler.

Datetime column converted to string for StringIndexer (Currently commented out because it is causing evaluations to overfit)

Problems:

Vector assembler missing columns 'local_resp', 'resp_ip_bytes', 'ts', 'datetime_str' as it's causing evaluations to overfit?

In [ ]:
spark.sparkContext.stop()